In [1]:
import os
import re
import pandas as pd
from collections import Counter
import sys
import requests
import json
import re
from zipfile import ZipFile
import streamlit as st

In [4]:
class LogAnalytics:
    def __init__(self):
        self.item_counts = None
        self.total_calls = 0
        self.valid_calls = 0
        self.total_states = 0
        self.class_count = 0
        self.call_drop = 0
        self.count_class = 0
        self.most_common_ngrams = []
        self.content = []
        self.files = []
        self.calls = []
        self.none_calls_1 = []
        self.none_calls_2 = []
        self.none_calls_3 = []
        self.state_str = []
        self.state_seq_call = []
        self.state_seq = []
        self.trans_list = []
        self.transcripts = []
        self.state_keywords = ['transferring call','ello', 'ntro', 'nterested', 'AGE', 'Age', 'ransfer', 'achine', 'reeting', 'reetings', 'itch', 'OPT', 'arrier', 'panish', 'DNC', 'dnc', 'busy', 'Busy', 'ositive', 'egative', 'XFER', 'ualifies', 'ualified']
        self.number_data = {}
        self.filings = str
        self.filers_name = []
        self.splitted_calls_3 = []
        self.path = "E://Python Practice//call_analytics_tool//uploaded_files//"
        #put keywrods here
        self.state_dict = {
            "playing hello": "hello",
            "playing intro": "intro",
            "playing Pitch OPT": "pitch",
            "No Answer": "no_answer",
            "Hang Up": "hang_up",
            "Caller Hanged Up": "caller_hanged_up",
            "Bot Hanged Up": "bot_hanged_up",
            "DNC": "dnc",
        }
        self.df = ""
        self.data =""
        self.mergerd_dict = {}
        self.new_dict= {}


    def zip_extractor(self,file_names):     
        # opening the zip file in READ mode
        for filing in file_names:
            with open(os.path.join(self.path,filing), 'r') as zip:
                # printing all the contents of the zip file
                zip.printdir()
            
                # extracting all the files
                # print('Extracting all the files now...')
                zip.extractall()
                # print('Done!')


    def fileReader(self,file_name):
        
        for filing in file_name:
            self.filings = filing
            try:
                with open(os.path.join(filing), "r") as file:
                    self.data = file.read()
                    # .decode("utf-8")
                    self.files.append(filing)
                    self.content.append(self.data)
            except FileNotFoundError:
                print(f"file not found: {filing}")
            except IOError:
                print(f"error occured while reading file: {filing}")


    def callSplitter(self):
        for callContent in self.content:
            if "call ended!!!" in callContent:
                splitted_calls = callContent.split("call ended!!!")
                for call in splitted_calls:
                    self.calls.append(call)


    def callCounter(self):
        self.total_calls = len(self.calls)
        return self.total_calls


    def getStates(self):
        for data in self.content:
            for line in data.splitlines():
                # check if state line not exists in state_str (prevent duplicates) then append it.
                if line not in self.state_str:
                    for kw in self.state_keywords:
                        if "----" in line and kw in line:   
                            self.state_str.append(line)


    def stateSequence(self):
        # getting all states from a call storing into list state_seq

        for call in self.calls:
            # print(call)
            state_seq_call = []
            for line in call.splitlines():
                # print(line)
                # check if iterated line exists in state's list. Then append it in sequence list
                if line in self.state_str:   
                    # print(line)     
                    state_seq_call.append(line.lower())
            # print(self.state_seq_call)
            self.state_seq.append(state_seq_call)
            # print(self.state_seq)
            # self.state_seq_call.clear()


    def new_states_counter(self,):
        # getting all states from a call storing into list state_seq
        for call in self.calls:
            state_seq_call = []
        
            for line in call.splitlines():
                # check if iterated line exists in state's list. Then append it in sequence list

                # get the phone number
                if "Incoming:" in line:
                    number = line.split(" ")[1]
                
                if line in self.state_str:  
                    line = line.replace("-","")
                    line = line.replace("playing","")
                    line = line.strip()  
                    state_seq_call.append(line.lower())

            self.new_dict[number] = state_seq_call 

    def countValidCalls(self):
        for call_sequence in self.state_seq:
            if len(call_sequence)>=2:
                if "achine" not in call_sequence[-1] and "DNC" not in call_sequence[-1] and "dnc" not in call_sequence[-1] and "ualified" not in call_sequence[-1]:
                        self.valid_calls+=1
            else:
                continue


    def countCallDrops(self, class_name):
        class_name = class_name.lower()
        for call_sequence in self.state_seq:
            if len(call_sequence)>=2:
                if class_name in call_sequence[-1]:
                    self.call_drop+=1
            else:
                continue  


    def countCallDrops_new(self,class_name):
        class_name = class_name.lower()
        for call_sequence in self.state_seq:
            if len(call_sequence)>=2:
                if class_name in call_sequence[-1]:
                    self.call_drop+=1
            else:
                continue  


    def countClass(self,class_name):
        for call_sequence in self.state_seq:
            for state in call_sequence:
                if class_name in state:
                    self.count_class+=1


    def countMostUsedPharses(self):
        
        for call_data in self.content:
            for line in call_data.splitlines():
                if "AI bot got this data =" in line:
                    trans_line = line.split("=")[-1]
                    self.trans_list.append(trans_line)
                    self.transcripts = list(set(self.trans_list))

        # WORKING TO EXTRACT MOST USED PHRASES

        # preprocess transcripts
        self.transcripts = [re.sub(r'[^\w\s]', '', transcript.lower()) for transcript in self.transcripts]

        # count n-grams
        n = 10 # length of phrase in words
        ngrams = Counter()
        for transcript in self.transcripts:
            words = transcript.split()
            ngrams.update(' '.join(words[i:i+n]) for i in range(len(words)-n+1))

        # get most common n-grams
        self.most_common_ngrams = ngrams.most_common(5)


    def none_separator_1(self,):
        data_lines = self.data.split('\n')
        results = []
        result = {}
        # "DNC", "Not Interested", "Ans Machine", "Transfer",
        states_to_find = ["DNC", "Not Interested", "Ans Machine","Hang Up","Not Qualified","Negative","Positive", "Transfer","Bot Hanged UP","No Answer","Caller Hanged Up"]
        phone_num = '123'
        for i in range(len(data_lines)):
            line = data_lines[i]

            # Detect phone number
            phone_num_match = re.search(r"Incoming: \((\d{3})\)(\d{3})-(\d{4})", line)
            if phone_num_match:
                phone_num = phone_num_match.group(0).split(':')[1].strip()
                result['Phone Number'] = phone_num_match.group(0).split(':')[1].strip()
            if '------------ playing' in line:
                line = line.replace("-","")
                line = line.replace("playing","")
                line = line.strip()
                result['Current State'] = line
            list_dat = ['No Answer','- DNC -','- Not Qualified -','- Not Interested -','- Ans Machine -','Transferring call']
            for dat in list_dat:
                if dat in line:
                    line = line.replace("-","")
                    line = line.strip()
                    result['Current State'] = line
            # Detect AI bot data
            ai_bot_match = re.search(r"AI bot got this data = (.*)", line)
            if ai_bot_match:
                result['AI bot got this data'] = ai_bot_match.group(1)

            # Detect AI bot level
            ai_bot_level_match = re.search(r"AI bot level= 1 : None", line)
            if ai_bot_level_match:
                result['AI bot level'] = ai_bot_level_match.group(0)
                for j in range(i+1, min(i+5, len(data_lines))):
                    next_line = data_lines[j]
                    if 'playing' in next_line:
                        continue
                j = i+1
                next_line = data_lines[min(j, len(data_lines)-1)]
                result['Next State'] = []
                token = False
                while 'call started' not in next_line:
                    for state in states_to_find:
                        if state in next_line:
                            result['Next State'] += [state]
                            break
                    if j >= len(data_lines):
                        break
                    j += 1
                    next_line= data_lines[min(j, len(data_lines)-1)]
                result['Phone Number'] = phone_num
                result['Next State'] = list(set(result['Next State']))    
                results.append(result.copy())  # Save this result
                result = {}  # Clear for next potential result
        
        if len(results) > 0:

            self.df_temp = pd.DataFrame(results)

            # st.dataframe(df)
            # df = df.dropna()
            # df.to_csv(f'{file_name[:-4]}_processed.csv', index=False)
        else:
            self.df_temp = pd.DataFrame()
            print(f"No results found in")


    def none_separator_2(self,):
        # Split the data by call start
        calls = self.data.split('call started')

        rows = []

        # For each call
        for call in calls[1:]:  # The first split is empty
            row = {}

            # Find phone number
            phone_num_match = re.search(r"Incoming: \((\d{3})\)(\d{3})-(\d{4})",call)
            if phone_num_match:
                phone_num = phone_num_match.group(0).split(':')[1].strip()
                row['Phone Number'] = phone_num_match.group(0).split(':')[1].strip()
            
            # States
            states = []
            for line in call.split("\n"):
                for state in self.state_dict:
                    if state in line:
                        states.append(self.state_dict[state])
            row['states'] = ', '.join(states)

            # Append to the rows
            rows.append(row)

        # Convert to DataFrame
        df = pd.DataFrame(rows)
 

        merged_df = self.df_temp.merge(df, on='Phone Number', how='inner')
        merged_dict_temp=merged_df.to_dict('records')
        self.mergerd_dict = merged_dict_temp


    def numberData(self):
        '''
        to extract number data ... 
        I made some changes and comment for that. 
        
        '''
        number_trans = []
        number_dis   = []
        numbers = []
       
        for call_index,call in enumerate(self.calls):

            number_transcript = [] #every time we make a transcript list
            if "AI bot got this data =" in call and "Incoming:" in call and "Disposition =" in call:
                splited_lines = call.splitlines()
                
                for line_index,line in enumerate(splited_lines):
                    if "Incoming:" in line:
                        number = line.split(" ")[1]
                        numbers.append(number)
                    if "AI bot got this data =" in line:
                        text = line.split("=")[1]
                        number_transcript.append(text)
                    if  "Disposition =" in line:
                        disposition = line.split("=")[1]
                        disposition = disposition[:-19] # remove the line containing slowing with 2sec etc (By. ALI)
                        number_dis.append(disposition)
                
                number_trans.append(number_transcript[0]) # getting only first Index of Transcript list

        # Display in One Dataframe
        try:
            df_number_data = {"file_id":self.filings,"Caller_ID":{self.filings:numbers}, "Transcript":{self.filings:number_trans},"states_number":{self.filings:self.new_dict}, "Disposition":{self.filings:number_dis}, "AI None Separater":{self.filings:self.mergerd_dict}, "total_calls":self.total_calls, "valid_calls":self.valid_calls, "total_states": self.count_class, "call_drop": self.call_drop }
            self.number_data = df_number_data       
        except:
            pass 


    def driver(self,files_name):
        class_name = "AGE Question"
        # self.zip_extractor(file_names)        
        self.fileReader(files_name)
        self.callSplitter()
        self.callCounter()
        self.getStates()
        self.stateSequence()
        self.countCallDrops(class_name)
        self.countValidCalls()
        self.countClass(class_name)
        self.countMostUsedPharses()
        self.none_separator_1()
        self.none_separator_2()
        self.new_states_counter()
        self.numberData()
        # self.df_temp = pd.DataFrame()
        # self.mergerd_dict = {}
        # self.total_calls = 0
        # self.valid_calls = 0
        # self.total_states = 0
        # self.class_count = 0
        # self.call_drop = 0
        # self.count_class = 0
        # self.most_common_ngrams = []
        # self.content = []
        # self.files = []
        # self.calls = []
        # self.state_str = []
        # self.state_seq_call = []
        # self.state_seq = []
        # self.trans_list = []
        # self.transcripts = []
        # self.df = ""
        # self.data =""
        # self.mergerd_dict = {}
        # self.filings = str
        # self.filers_name = []
        # self.splitted_calls_3 = []
        # self.new_dict= {}

        return self.number_data
    
    

files_name = ["7C-D3-0A-1A-C3-C4_1676679530.txt"]
interface = LogAnalytics()
interface.driver(files_name)

{'file_id': '7C-D3-0A-1A-C3-C4_1676679530.txt',
 'Caller_ID': {'7C-D3-0A-1A-C3-C4_1676679530.txt': ['(573)434-4981',
   '(215)515-1792',
   '(618)204-2481',
   '(956)660-6135',
   '(918)967-4208',
   '(412)766-0467',
   '(719)374-0829',
   '(309)228-1048',
   '(309)652-3889',
   '(304)536-9363',
   '(337)294-8660',
   '(410)437-9891',
   '(803)871-2812',
   '(626)578-7147',
   '(337)738-2103',
   '(817)240-7437',
   '(651)293-1352',
   '(251)250-5601',
   '(361)564-3461',
   '(408)702-8621',
   '(501)282-8681',
   '(856)488-6150',
   '(218)356-2610',
   '(804)349-4745',
   '(662)285-7763',
   '(563)249-3913',
   '(281)839-8072',
   '(210)589-2842',
   '(612)393-4692',
   '(201)567-8367',
   '(601)382-4376',
   '(972)998-0270',
   '(574)896-2057',
   '(601)207-5055',
   '(773)701-6367',
   '(717)553-7747',
   '(704)201-7159',
   '(817)479-6844',
   '(619)867-1043',
   '(773)990-0897',
   '(712)217-7505',
   '(956)420-1620',
   '(918)241-5777',
   '(864)280-5312',
   '(415)240-6538',
   